#                                       **CIFAR-10: Image Classification**

#   Step 1: Data Preprocessing & Loading 
##  Visualization of Images and Labels / Inserting Grayscale Conversion / Augmentation 


%pip install matplotlib
%pip install numpy
%pip install tensorflow
%pip install tensorflow-gpu

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, RandomFlip, RandomRotation, Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.utils import to_categorical

In [7]:
# Load the CIFAR-10 Dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

In [8]:
# Check data dimensions
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [9]:
# Define a list with all the class labels for CIFAR-10
classes = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

# Function to visualize color images from CIFAR-10 dataset with correct labeling
def visualize_color_images_with_labels(images, labels, classes, images_per_class=10, title="CIFAR-10 Images"):
    num_classes = len(classes)
    total_images = num_classes * images_per_class

    plt.figure(figsize=(6, 6))
    image_count = 0

    # Loop through class labels to pick images_per_class images per class
    for class_index, class_name in enumerate(classes):
        class_images = images[labels.flatten() == class_index][:images_per_class]

        # Loop through the images, arranging them dynamically
        for img in class_images:
            plt.subplot(num_classes, images_per_class, image_count + 1)
            plt.imshow(img)
            plt.axis('off')
            
            # Add class label to the left side of each row
            if image_count % images_per_class == 0:
                plt.text(-30, 32 // 2, class_name, rotation=0, size='large', va='center', ha='right')
            
            image_count += 1
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

# Visualize color images from the CIFAR-10 training set
visualize_color_images_with_labels(x_train, y_train, classes, images_per_class=10, title="CIFAR-10 Training Images")


In [10]:
# Data Augmentation:

# Convert images to grayscale

grayscale_x_train = tf.image.rgb_to_grayscale(x_train)
grayscale_x_test = tf.image.rgb_to_grayscale(x_test)

gray_x_train = np.array(grayscale_x_train)
gray_x_test = np.array(grayscale_x_test)

print(gray_x_train.shape)
print(gray_x_test.shape)

# Create augmentation layer for model (used further down)

data_augmentation = Sequential([
layers.RandomFlip("horizontal_and_vertical"),
layers.RandomRotation(0.2),
]) 


In [11]:
# Normalize the images to the range [0, 1]
x_train_normalized = gray_x_train.astype('float32') / 255.0
x_test_normalized = gray_x_test.astype('float32') / 255.0

print(x_train_normalized.shape)
print(x_test_normalized.shape)

In [12]:
# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

print(y_train.shape)
print(y_test.shape)

Task, Diego:
Transfer Traning (VGG-16 can work well, imagenit, inseption, densnet, resnet) Check which one is the most efficient to clasify our image model.
Build a model Densnet
- Research different networks to see what kind of data they were trained on (image classes, how many...?)
- Decide on best one for our dataset
- Think about how many layers to add on top of that for our specific model
- Think about which layers to freeze/ unfreeze when training with the new layers
- Adjust epochs, other parameters related to our new model which could optimize

In [18]:
# TO DO:
# Try different optimizer (RMSProp)
# Try bigger model (10-15 layers (2-3 Conv. layers, max pooling)) -> similar to VGG 16?
# Second last layer: 10 nurons, maybe try softmax for second last layer too
# Different loss function (try focal loss)?
# Insert batch normalization layers 
# Play around with number of neurons, batch size and epochs
# Try with non-augmented images
# Note what gives good results!

# Finalize code for confusion matrix and visualizing loss and accuracy functions (see Step 4 below)


# Define model / data parameters
num_classes = 10
input_shape = x_train_normalized.shape[1:]
dropout_rate = 0.2
epochs = 10
batch_size = 32

# Define Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define custom optimizer, learning rate
optimizer = Adam(learning_rate=0.001)

# Perform the train-validation split
x_train_normalized_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train_normalized, y_train, test_size=0.2, random_state=42)

# Define the model with data augmentation
model = Sequential([
    layers.Input(shape=input_shape),
    data_augmentation,  # Data augmentation layer
    layers.Conv2D(32, (3, 3), padding="same", activation='relu'),
    #BatchNormalization(),
    layers.Conv2D(32, (3, 3), padding="same", activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2), padding="same"),
    layers.Conv2D(64, (3, 3), padding="same", activation='relu'),
    #BatchNormalization(),
    layers.Conv2D(64, (3, 3), padding="same", activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2), padding="same"),
    layers.Flatten(),
    layers.Dense(50, activation='relu'),
    layers.Dropout(dropout_rate),
    layers.Dense(10, activation='softmax'),
    layers.Dropout(dropout_rate),
    layers.Dense(num_classes, activation='softmax')
])

# Print summary of the model
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32, 32, 1)         0         
                                                                 
 conv2d_10 (Conv2D)          (None, 32, 32, 32)        320       
                                                                 
 conv2d_11 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 16, 16, 64)        18496     
                                                                 
 conv2d_13 (Conv2D)          (None, 16, 16, 64)        36928     
                                                      

In [19]:
# Compile the model
model.compile(optimizer = optimizer,
              loss ='categorical_crossentropy',
              metrics = ['accuracy'])

# Train the model with normalized data
history = model.fit(x_train_normalized, y_train, validation_data=(x_val_split, y_val_split), epochs = epochs, batch_size = batch_size, callbacks = [early_stopping])


Epoch 1/10
1563/1563 [==============================] - 135s 85ms/step - loss: 2.2684 - accuracy: 0.1368 - val_loss: 2.2041 - val_accuracy: 0.1900
Epoch 2/10
1223/1563 [======================>.......] - ETA: 31s - loss: 2.2252 - accuracy: 0.1678

KeyboardInterrupt: 

#  Step 4: Model Evaluation
## Evaluate the Model and Compute Metrics

In [4]:
# Print training accuracy and loss curves
print(history.history.keys())

print(history.history['loss']) # returns the loss value at the end of each epoch
print(history.history['accuracy']) # returns the accuracy at the end of each epoch

plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')

plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='green', label='train')
plt.show()

NameError: name 'history' is not defined

In [ ]:
# Make prediction
predictions = model.predict(x_test)

predictions = np.argmax(predictions, axis=1)

# Plot confusion matrix
from sklearn.metrics import confusion_matrix

gt = np.argmax(y_test, axis=1)
confusion_matrix(gt, predictions)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_train, annot=True, fmt='d', cmap='Blues',
            xticklabels=data.target_names,
            yticklabels=data.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for the Training Set')
plt.show()


conf_matrix_test = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix for the Testing Set:")
print(conf_matrix_test)

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_test, annot=True, fmt='d', cmap='Blues',
            xticklabels=data.target_names,
            yticklabels=data.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for the Testing Set')
plt.show()

In [ ]:
# Print test accuracy and test loss for trained model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)